In [2]:
import numpy as np
import pandas as pd 

In [3]:
df = pd.read_csv("D:\\covid_toy.csv")
df.head(10)
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [4]:
from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(df.drop(['has_covid'],axis=1),
                                                       df['has_covid'],
                                                       test_size=0.3)
x_train.head(10)

,age,gender,fever,cough,city
51,11,Female,100.0,Strong,Kolkata
37,55,Male,100.0,Mild,Kolkata
82,24,Male,98.0,Mild,Kolkata
22,71,Female,98.0,Strong,Kolkata
96,51,Female,101.0,Strong,Kolkata
56,71,Male,NaN,Strong,Kolkata
69,73,Female,103.0,Mild,Delhi
70,68,Female,101.0,Strong,Delhi
68,54,Female,104.0,Strong,Kolkata
39,50,Female,103.0,Mild,Kolkata


In [5]:
y_train.head(10)

51    Yes
37     No
82    Yes
22    Yes
96    Yes
56     No
69     No
70     No
68     No
39     No
Name: has_covid, dtype: object

In [6]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_train_fever.shape

(70, 1)

In [7]:
x_test_fever = si.fit_transform(x_test[['fever']])
x_test_fever.shape

(30, 1)

In [8]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_oe = oe.fit_transform(x_train[['cough']])
x_test_oe = oe.transform(x_test[['cough']])
x_train_oe.shape


(70, 1)

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse_output=False)
x_train_ohe = ohe.fit_transform(x_train[['gender','city']])
x_test_ohe = ohe.transform(x_test[['gender','city']])
x_train_ohe.shape

(70, 4)

In [10]:
x_test_ohe.shape

(30, 4)

In [11]:
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values
x_train_age.shape

(70, 1)

In [12]:
x_test_age.shape

(30, 1)

In [13]:
x_train_transformed = np.concatenate((x_train_age,x_train_ohe,x_train_oe,x_train_fever),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_ohe,x_test_oe,x_test_fever),axis=1)
x_train_transformed.shape

(70, 7)

In [14]:
x_test_transformed.shape

(30, 7)

In [15]:
from sklearn.compose import ColumnTransformer

In [16]:
transformer = ColumnTransformer(transformers=[
        ('tnf1',SimpleImputer(),['fever']),
        ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
        ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [20]:
x_train_trans_1 = transformer.fit_transform(x_train)

In [21]:
x_test_trans_1 = transformer.transform(x_test)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [23]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()

In [24]:
lr.fit(x_train_trans_1,y_train)
dt.fit(x_train_trans_1,y_train)

D:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


DecisionTreeClassifier()

In [26]:
y_predict_1 = lr.predict(x_test_trans_1)

In [28]:
y_predict_2 = dt.predict(x_test_trans_1)

In [27]:
from sklearn.metrics import accuracy_score , confusion_matrix

In [29]:
accuracy_score(y_test,y_predict_1)

0.4

In [30]:
accuracy_score(y_test,y_predict_2)

0.43333333333333335